In [134]:
import numpy as np 
import pandas as pd 
import plotly.express as px  # (version 4.7.0)
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio 
from datetime import datetime
pio.renderers.default='browser'

In [304]:
def create_data_density(dep_pop_file,dep_sup_file):
    dep_sup = pd.read_csv(dep_sup_file,dtype={'dep': str, 'code': str, 'Superficie(km2)': float})
    dep_pop = pd.read_csv(dep_pop_file,dtype={'dep': str, 'code': str, 'Population(milliers)': float})
    dep_sup = dep_sup.sort_values(by=['code'])
    dep_pop = dep_pop.multiply(1000, axis='Population(milliers)')
    dep_sup['Population(hab)'] = (dep_pop['Population(milliers)'].values)
    dep_sup['Density(hab/km2)'] =round(dep_sup['Population(hab)']/dep_sup['Superficie(km2)'],2)
    dep_sup['Density(hab/km2)_log'] = np.log10(dep_sup['Density(hab/km2)'])
    return dep_sup
def create_data_covid(covid_file,dep_pop_file,dep_sup_file): 
    data= create_data_density(dep_pop_file,dep_sup_file)
    covid = pd.read_csv(covid_file)
    covid = pd.merge(data, covid, on = ['code'])
    for i in ['974','973','972','971']: 
        covid= covid.drop(covid[ covid['code'] ==i].index, axis=0)
    covid['sum']= covid['hosp'] + covid['rea']
    covid['sum']=covid['sum'].astype('float64')
    for i in range(len(covid['sum'])): 
        if np.log(covid['sum'][i])==float('-inf'):
            covid['sum'][i]=1
    covid['jour'] =pd.to_datetime(covid['jour']).dt.date
    covid = covid.set_index(pd.DatetimeIndex(covid['jour']))
    
    return covid



In [305]:
covid = create_data_covid("Data/donnees-hospitalieres-covid.csv","Data/dep-pop.csv","Data/dep-sup.csv")
france = json.load(open("Geojson/france_dep.geojson",'r'))

In [190]:
d = {int(i):str(j) for i,j in zip(range(len(covid.index.unique())), covid.index.unique())}

In [213]:
d2

{14: 'Avril',
 44: 'Mai',
 75: 'Juin',
 105: 'Juillet',
 136: 'Aout',
 167: 'Septembre',
 197: 'Octrobre'}

In [202]:
mois = {'01':'Janvier','02':'Février','03':'Mars','04':'Avril','05':'Mai','06':'Juin','07':'Juillet','08':'Aout','09':'Septembre','10':'Octrobre','11':'Novembre','12':'Decembre'}
def transform(date): 
    tmp = date.split('-')
    s = tmp[2] +' '+mois[tmp[1]]+' '+tmp[0]
    return s 

transform(d[24])

11 Avril 2020


In [306]:
# get specific data for 1 sexe dans 1 day
covid = covid[covid['sexe']==2]
covid = covid.loc[datetime.strptime(d[0], '%Y-%m-%d').date()]
covid = covid.sort_values(by=['Density(hab/km2)'])

In [300]:
covid[covid['sum']==0]

,dep,code,Superficie(km2),Population(hab),Density(hab/km2),Density(hab/km2)_log,sexe,hosp,rea,rad,dc,sum
jour,,,,,,,,,,,,


In [307]:
#ig = go.Figure()

fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.35, 0.65],
    horizontal_spacing=0.03,
    specs=[[{"type": "Choroplethmapbox"}, {"type": "bar"}]],
    subplot_titles=("Hospitalisation et Réanimation par département en France","Hospitalisation et Réanimation par département en France en fonction de la densité"))

fig.add_trace(go.Bar(
    x=covid['dep'],
    y=covid['hosp'],
    name='Hospitalisation',
    hovertemplate ="<b> Departement : %{customdata[0]} </b><br><b> Densité : %{customdata[4]} hab/km2</b><br><b>Hospitalisation : %{customdata[7]} pers.</b>"+"<extra></extra>",
    customdata=covid,
    marker_color='rgb(196, 102, 73)',
    showlegend =False),
    row=1, col=2)

fig.add_trace(go.Bar(
    x=covid['dep'],
    y=covid['rea'],
    name='Réanimation',
    hovertemplate ="<b> Departement : %{customdata[0]} </b><br><b> Densité : %{customdata[4]} hab/km2</b><br><b>Réanimation : %{customdata[8]} pers.</b>"+"<extra></extra>",
    customdata=covid,
    marker_color='rgb(89, 13, 31)',
    showlegend =False),
    row=1, col=2)

fig.add_trace(go.Choroplethmapbox(geojson=france, 
                                    locations=covid['code'],
                                    z=np.log10(covid['sum']),
                                    customdata=covid,
                                    showscale=False,
                                    hovertemplate ="<b> Departement : %{customdata[0]} </b><br><b> Densité : %{customdata[4]} hab/km2</b><br><b>Hospitalisation : %{customdata[7]} pers.</b><br><b>Réanimation : %{customdata[8]} pers.</b>"+"<extra></extra>",
                                    featureidkey="properties.code",                           
                                    colorscale="amp",
                                    showlegend = False,
                                    marker_opacity=0.8, 
                                    marker_line_width=0.3,
                                    name='europe_map'),
                                    row=1, col=1)

fig.update_layout(title = go.layout.Title(text = 'Density Urbaine et covid'),mapbox_style="carto-positron",mapbox_zoom=4.7, mapbox_center = {"lat": 46.35, "lon": 2.55})

fig.update_layout(barmode='relative')
fig.update_layout(template='plotly_white')

fig.show()